## Description：
这节课，我们的任务是做一个快手用户的活跃预测。假如我们拿到了一份快手平台记录的关于快手用户的30天一个行为数据集(记录用户注册，登录，视频观看与发布，互动的记录），我们的目标就是根据这个行为数据集预测在未来七天的活跃用户<br><br>

任务分为如下几部分：
>* 任务目标与数据分析
>* 整体模型架构
>* 构建用户特征序列, 生成汇总表
>* 制作标签
>* 建立模型并训练
>* 得出最终的模型结果
>* 知识点和思路总结

In [2]:
"""先导入包"""
import numpy as np
import pandas as pd
import datetime

import tensorflow as tf
from deep_tools import f
from deep_tools import DataGenerator

## 任务目标与数据分析
在这一部分，我们先看看拿到的是一份什么样的数据，然后分析一下我们这个任务到底怎么应该去做。
<br><br>我们会拿到四个表格，分别是：
* 用户注册日志表，记录的是用户什么时候使用什么方式（微信，qq）什么设备（手机，pad）进行注册的快手账号，会有四个字段的信息，注意，day这一列，由于我们搜集的是30天的数据，所以这里的取值是1-30， 表示的是用户第几天的行为，后面那几份数据也是一样。<br>
![](img/1.png)
<br>
* APP启动日志表，记录的是用户在第几天登录过快手app， 两个字段的信息<br>
![](img/2.png)
<br>
* 视频创建日志表， 记录的是用户在第几天创作过视频 <br>
![](img/3.png)
<br>
* 用户行为日志表，这个比较关键，也是我们后面制作特征的主要渠道来源，由用户的行为才能看出用户是不是活跃<br>
![](img/4.png)
<br>

下面，我们就导入数据，详细的看一下这些数据吧

In [2]:
"""读取数据集"""
register = pd.read_csv('user_register_log.txt', sep='\t', names=['user_id', 'register_day', 'register_type', 'device_type'])
register.head()   # 注意这里的register_day 不一定都是1， 1代表我们搜集的30天里面，这个是第1天就注册了，还有的不是第一天注册的，后面会分析。

,user_id,register_day,register_type,device_type
0,744025,1,1,283
1,1270299,1,1,259
2,571220,1,1,2
3,1308501,1,0,23
4,745554,1,2,0


In [3]:
launch = pd.read_csv('app_launch_log.txt', sep='\t', names=['user_id', 'launch_day'])
create = pd.read_csv('video_create_log.txt', sep='\t', names=['user_id', 'create_day'])
activity = pd.read_csv('user_activity_log.txt', sep='\t', names=['user_id', 'act_day', 'page', 'video_id', 'author_id', 'act_type'])
activity.head()

,user_id,act_day,page,video_id,author_id,act_type
0,1062323,22,3,2877472,880271,0
1,639898,17,3,740662,210200,0
2,1260200,5,3,3332414,162866,0
3,817201,22,3,1129617,530246,0
4,817201,23,3,1129617,530246,0


In [4]:
launch.head()   # 记录用户第几天登录过

,user_id,launch_day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


In [5]:
create.head()   # 记录用户哪一天创建过视频信息

,user_id,create_day
0,720497,1
1,720497,1
2,720497,1
3,1075211,6
4,1075211,12


下面，我们开始对这个数据进行一个解析了， 我们拿到的是一个连续的30天之内用户的一个行为数据，包含了四个数据集，分别记录用户的注册，登录，创建视频，使用行为的信息， 但是有一个问题要注意，就是不一定每个用户的信息都是30天的记录，因为，我们这里有一个注册信息表，里面有一个register_day的字段，这个字段不一定所有用户都是1，即不一定每个用户都是在第一天进行注册的。比如，假设一个用户在第七天才注册的，那么前七天就没有这个用户的信息，假设一个用户是在第25天注册的，那么这个用户只有后五天的数据。 所以这是这个任务的一个难点所在，就是**每个用户的序列长度并不是一样的**， 所有后面我们分析的时候，对于一个用户，先分析是在哪一天注册的，我们再从这一天开始去算。

<br><br> 
再就是我们任务进行一个解析，我们拿到了每个用户的这样的数据，是要预测未来的活跃度，注意这个未来，不是接下来的一两天，而是挺长的一段时间，一个周期这样的。我们这里是预测未来七天内的， 比如有个用户第七天注册了，我们预测的是他第七天到第十四天的活跃度的可能性。这就是我们的目标，基于之前的一段序列数据，来预测未来七天的活跃度的可能性。

## 整体模型架构
这一块，主要是介绍最终介绍的模型长什么样子，根据这个模型，去倒推我们需要去构建什么样的特征和标签（这其实也是一种考虑问题的方式，之前我们可能都是拿到数据，然后特征工程，然后根据我们制作的特征和标签去构建一系列模型，进行预测分类等。 但是有时候我们拿到一个任务之后，或许一下子不太明白究竟要做什么样的数据出来，尤其是这种实际的任务分析。 ），这时候，我们不妨先从模型的角度去考虑任务，先看看什么样的模型适合解决这个任务，然后我们假设已经有了这样的一个模型，那么我们应该考虑这个模型要什么样的输入， 这样，我们再去做相应的输入出来，这也是考虑问题的一种方式吧。 所以这次先从模型开始。<br><br>
关于模型的问题，我们做的虽然说预测用户的活跃度，但可以转化成一个二分类的问题，最后输出就是未来七天时间内，这个用户是否活跃，0或者1. 既然是二分类的问题，那么这里的模型选择就比较多了，什么xgb，lgb，这些优秀的机器学习算法都可以，但是我们分析一下，我们最后拿到的这个数据应该是针对某一个用户，会有从他注册到后面行为的一系列信息，而这些都是时间上进行相关的，比如一个用户第七天注册了，那么我们就有这个用户第七天到第30天的各种行为，也是一种时间序列的预测。关于这种时间序列的预测，如果要使用机器学习的方式，就需要用滑动窗口去切割数据集，并且特征提取上需要使用历史的统计量。<br><br>我们这里使用神经网络的方式，也就是RNN来完成这个任务，这样不需要对输入序列做过多处理，也不使用滑动窗口。毕竟时间的监督学习任务，RNN还是比较擅长的，能够捕捉到时间的这种关联信息。 <br><br>
使用RNN，一般地会想到如下解决方案：以几天内的用户行为序列为输入，以未来七天该用户是否活跃为标签，标注该序列。这是一种Many2One的解决方案。但是这样做的话为了充分利用数据，需要对训练数据做大量的滑窗，以实现数据增广，计算成本高。另外，每个序列只有一个标签，梯度难以传导，导致训练困难。所以我们可以考虑Many2Many结构，即每个输入都对应输出之后7天是否活跃，充分利用监督信息，减轻梯度传到负担，使训练更加容易。我们就可以构建出RNN的大体模型长这样：
![](img/6.png)
<br><br> 这样我们的网络架构就构思出来了，但是这里还有一个细节，就是序列是变长的，就是用户的记录天数是不同的，每个batch中取相同长度的序列，不同batch长度不同，每次随机取某一长度的batch。这时候我们需要使用动态RNN，根据时间序列的长度自己调整。 

那么我们怎么去构建数据呢？ 我们知道上面的都是一些label值，也就是0或者1的这些，表示活跃不活跃。 下面的序列就是我们的输入数据，但是t1,t2,...tn这些数据应该是长什么样子呢？  

## 构建用户特征序列
这一块，是我们的重点。我们就是要看看这个用户特征序列究竟怎么样才能构建出来。 思路如下：
>1. 保存每个用户的记录数据
>2. 创建用户字典
>3. 构造用户特征序列


### 保存每个用户的记录数据
既然之前我们说过，每个用户的记录序列长度是不一样子的，所以这个地方我们应该首先先统计每个用户到底有几天的记录数据，把这个长度我们先保存下来。这个比较简单，直接用总天数减去他注册的时间就可以啦。

In [4]:
"""计算序列长度： 持续时间 = 数据总时间 - 注册时间"""
register['seq_length'] = 31 - register['register_day']
register.head()

,user_id,register_day,register_type,device_type,seq_length
0,744025,1,1,283,30
1,1270299,1,1,259,30
2,571220,1,1,2,30
3,1308501,1,0,23,30
4,745554,1,2,0,30


### 创建用户记录字典
根据前面的记录天数，创建一个字典，来存储不同记录天数的用户到底有哪些, key表示记录长度，value表示用户id。
比如只有1天记录的有哪些用户，2天记录的有哪些，...30天完整记录的用户有哪些），这个的目的就是因为我们的用户序列都不等长，也就是每个用户的记录天数不一定一样，这样就导致如果统一喂入神经网络的话，没法组成一个矩阵的形式， 但如果根据记录天数把用户给分开了，那么同一个记录天数下会有很多用户，这些序列是等长的， 那么我们就可以把这些划分成一个batch给神经网络，就好处理了。就是网络的输入接收的每个batch的序列长度要求是相同的，但是不同batch的序列可以不同。

In [5]:
"""根据前面的记录天数，创建一个字典，来存储不同记录天数的用户到底有哪些 """
user_queue = {i: [] for i in range(1,31)}

for index, row in register.iterrows():   # 这个iterrows是对DataFrame进行行遍历，是在数据框中的行进行迭代的一个生成器，它返回每行的索引及一个包含行本身的对象。
    user_queue[row[-1]].append(row[0])     # row[-1]是seq_length, row[0]是user_id

### 构建用户特征
这一块，我们得考虑究竟怎么去提取特征呢？ 首先，得有一个特征的个数，就是针对每个用户，每天的记录，我们得提取多少个特征。 这些特征就是用户的行为，比如什么时候登陆，登陆之后，点没点赞，创没创建视频这些东西（都从数据集里面提取）。 所以对于一个用户来说，我们的输入数据最后应该是一个矩阵，每一行代表着记录的天数，每一列代表一个特征，看下面的图也知道为什么之前要统计不同记录对应的用户了吧，每个用户对应一个矩阵，然后很多个用户合起来就是一个3维的向量，这一个至少是一个立方体的形式。 如果用户的记录不一样的话，就没法进行用户的合并，所以我们这里有了这样一个字典之后，我们就可以把每个记录的用户看成一个batch放入神经网络了。
![](img/7.png)<br>
这一块的思路呢？  我们是要构建用户特征，所以我们定义一个用户序列的类，把所有相关的操作都放到用户序列的里面
> 1. 首先，会构建出每个用户的特征矩阵，就是上面的那种矩阵构建出来，初始化为0， 这里假设特征列为15，即f=15
> 2. 遍历上面的数据表，开始提取相应的特征，把这些特征填入相应用户的相应记录的相应特征中

In [21]:
"""定义一个user_seq类"""
class user_seq:
    
    def __init__(self, register_day, seq_length, n_features):
        """
            register_day: 用户第几天进行的登录
            seq_length: 用户序列的长度，就是记录了几天登录信息， 行数
            n_features: 每天提出的特征个数， 列数
        """
        self.register_day = register_day
        self.seq_length = seq_length
        self.array = np.zeros([seq_length, n_features])   # 这就是上面那个用户对应的矩阵形式，初始化位0
        self.page_rank = np.zeros([self.seq_length])
        self.pointer = 1
    
    # 提取特征填入特征矩阵
    def put_feature(self, feature_number, string):
        for i in string.split(','):
            pos, value = i.split(':')     # 注册后的第几天进行了登录，1为指示符
            self.array[int(pos)-self.register_day, feature_number] = 1   # 从注册后开始记录
    
    
    def get_array(self):
        return self.array
    
    # 得到标签  如果一个用户在未来七天活跃了，那么标记为1
    def get_label(self):
        self.label = np.array([None] * self.seq_length)    # 一个seq_length长度的数组
        active = self.array[:, :12].sum(axis=1)          # 这里选了一部分特征做了个sum，意思是不管是转发，登录，啥的，只要做了就算一次活动
        for i in range(self.seq_length-7):      # 这地方得控制一下，如果一个用户15-30的数据，那么我们标签最多只能到23天，因为30天之后的数据我们没有
            self.label[i] = 1 * (np.sum(active[i+1:i+8]) > 0)    # 这里对于当前的i，如果未来七天内活跃过，那么标签就是1
        
        return self.label          

#### 创建每个用户的二维矩阵，初始化为0

In [22]:
"""我们创建用户的那个记录矩阵"""
n_features = 15
data = {row[0]:user_seq(register_day=row[1], seq_length=row[-1], n_features=n_features) for index, row in register.iterrows()}

In [10]:
# 这样就得到了每个用户的初始特征表
data

{744025: <__main__.user_seq at 0x219001ba908>,
 1270299: <__main__.user_seq at 0x219001ba630>,
 571220: <__main__.user_seq at 0x219001ba8d0>,
 1308501: <__main__.user_seq at 0x219001ba828>,
 745554: <__main__.user_seq at 0x219001bad30>,
 1031012: <__main__.user_seq at 0x219001bada0>,
 913297: <__main__.user_seq at 0x219001ba898>,
 266500: <__main__.user_seq at 0x219001badd8>,
 475120: <__main__.user_seq at 0x2197fba4c18>,
 547944: <__main__.user_seq at 0x2197fba4e10>,
 916655: <__main__.user_seq at 0x2197fba4e48>,
 719262: <__main__.user_seq at 0x2197fba4f60>,
 1026175: <__main__.user_seq at 0x2197fba4f98>,
 1140342: <__main__.user_seq at 0x2197fba4da0>,
 688100: <__main__.user_seq at 0x2197fba4c50>,
 1342459: <__main__.user_seq at 0x2197fba4be0>,
 926263: <__main__.user_seq at 0x2197fba44e0>,
 40710: <__main__.user_seq at 0x2197fba4fd0>,
 246954: <__main__.user_seq at 0x2197fba4ba8>,
 153579: <__main__.user_seq at 0x2197fba45f8>,
 161418: <__main__.user_seq at 0x21900661278>,
 649526:

#### 然后，就是从数据表中提出相应的信息，填入特征矩阵
这一块，我们基于之前的数据表提取特征了，也是分为这几块提取
* 用户的登录信息特征(launch表)
* 用户创作视频的信息特征(create表)
* 用户使用时的行为特征，例如点赞，转发等（activity表的act_type）
* 用户产生行为的界面信息特征(activity表的page字段)
* 用户观看其他用户的行为特征(activity表的author_id字段 != user_id)
* 观看自己作品的行为特征(activity表的author_id字段 == user_id)

##### 用户的登录信息特征 （是否在某天登录过）- launch表

In [11]:
# 看看launch表长啥样来
launch.head()

,user_id,launch_day
0,383135,1
1,330986,4
2,330986,9
3,330986,11
4,330986,12


In [23]:
launch['launch_freq'] = 1    # 每个用户每天登录次数
launch_table = launch.groupby(['user_id', 'launch_day'], as_index=False).agg({'launch_freq':'sum'})
launch_table.head()
# 如果这里的as_index为true， 那么这里的索引就自动变成第一列，也就是user_id，这五个会用共同的索引。  如果是False，那就是SQL风格的输出

,user_id,launch_day,launch_freq
0,16,13,1
1,16,14,1
2,16,15,1
3,16,18,1
4,16,19,1


In [24]:
# 然后得根据用户的id进行整合一下，把相同用户的结果放在一块
def record_to_sequence(table):    # 得到用户特征序列表
    table.columns = ['user_id', 'day', 'value']
    table.sort_values(by=['user_id', 'day'], inplace=True)
    table['string'] = table.day.map(str) + ":" + table.value.map(str)
    table = table.groupby(['user_id'], as_index=False).agg({'string':lambda x:','.join(x)})
    return table

In [25]:
launch_table = record_to_sequence(launch_table)
launch_table.head()

,user_id,string
0,16,"13:1,14:1,15:1,18:1,19:1,20:1,21:1,22:1,23:1"
1,30,24:1
2,98,16:1
3,105,"12:1,14:1,15:1,16:1,17:1,18:1,19:1,20:1,21:1,2..."
4,176,"27:1,28:1,29:1,30:1"


In [26]:
"""然后我们把用户的登录信息填入到特征矩阵，是否登录的信息是第1列特征"""
# 例如ID=16的用户会在其特征表的指定位置的（13,14,15,18...行第1列进行填充）
for index, row in launch_table.iterrows():   # 根据登录信息对用户特征表进行填充
    data[row[0]].put_feature(0, row[1])      # row[0]就是用户ID， data[row[0]]就是用户对应的特征表，用put_feature函数将row[1],填入到第一列中

##### 创作视频的特征（是否在某天创建过视频） - create表

In [16]:
# 先看一下那个表的信息
create.head()

,user_id,create_day
0,720497,1
1,720497,1
2,720497,1
3,1075211,6
4,1075211,12


In [27]:
# 这个和上面的思路差不多，也是先进行统计，然后整合填充到第2列的特征
create['create_freq'] = 1
create_table = create.groupby(['user_id', 'create_day'], as_index=False).agg({'create_freq':'sum'})
create_table = record_to_sequence(create_table)
for index, row in create_table.iterrows():
    data[row[0]].put_feature(1, row[1])

##### 用户使用时的行为特征(activity表的act_type字段)
这一块根据是否有act_type字段的六种行为（播放，关注，点赞，转发，举报，减少此类作品）可以做出6个特征

In [28]:
# 看一下表中的信息
activity.head()
set(activity['page'])

{0, 1, 2, 3, 4}

In [29]:
"""这里使用的act_type字段，对于不同的行为，制作不同的特征，构建6个特征"""
for i in range(6):
    act_table = activity[activity.act_type==i].copy()    # 这个地方要深复制一下，不要动用原来的表
    act_table = act_table.groupby(['user_id', 'act_day'], as_index=False).agg({'video_id':'count'})
    act_table = record_to_sequence(act_table)
    for index, row in act_table.iterrows():
        data[row[0]].put_feature(i+2, row[1])

##### 用户产生行为的界面信息特征(activity表的page字段)

In [30]:
 for i in range(5):     
    act = activity[activity.page==i].copy()
    act = act.groupby(['user_id', 'act_day'], as_index=False).agg({'video_id':'count'})
    act = record_to_sequence(act)
    for index, row in act.iterrows():
        data[row[0]].put_feature(i+8, row[1])

##### 用户观看其他用户的行为特征(activity表的author_id字段 != user_id)

In [31]:
watched = register.loc[:, ['user_id']].copy()
watched.columns = ['author_id']
watched = pd.merge(watched, activity[activity.author_id != activity.user_id], how='inner')  # 只得到交集 相当于看别人视频的
watched = watched.groupby(['user_id', 'act_day'], as_index=False).agg({'video_id':'count'})
watched = record_to_sequence(watched)
for index, row in watched.iterrows():
    data[row[0]].put_feature(13, row[1])

##### 观看自己作品的行为特征(activity表的author_id字段 == user_id)

In [32]:
watched = activity[activity.author_id == activity.user_id].copy()
watched = watched.groupby(['user_id', 'act_day'], as_index=False).agg({'video_id':'count'})
watched = record_to_sequence(watched)
for index, row in watched.iterrows():
    data[row[0]].put_feature(14, row[1])

## 制作数据标签
通过上面的操作，我们就构建好了神经网络的输入，当然这里这样构建特征是为神经网络服务的，如果是想用机器学习的一些方式，就可以提取一些统计的特征，比如平均值，方差，中位数的这些东西用xgb什么的跑，但是这种时间之间的特征可能就捕捉不到了。<br><br>
下面，我们就是制作数据的标签了： 在未来七天内是否会使用APP， 这个的思路就是对用户从注册开始就进行统计，对于每1天的数据展开，如果其未来七天内这个用户活跃过，就标记为1

In [33]:
label = {user_id:user.get_label() for user_id, user in data.items()}
label

{744025: array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, None, None, None, None, None, None, None], dtype=object),
 1270299: array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, None, None, None, None, None, None, None], dtype=object),
 571220: array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, None, None, None, None, None, None, None], dtype=object),
 1308501: array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, None, None, None, None, None, None, None], dtype=object),
 745554: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, None, None, None, None, None, None, None], dtype=object),
 1031012: array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, None, None, None, None, None, None, None], dtype=object),
 913297: array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1

最后，把每一个用户对应的矩阵拿出来放入data， 特征制作完毕

In [34]:
data = {user_id: user.get_array() for user_id, user in data.items()}

In [37]:
data[744025].shape

(30, 15)

## 建立模型并且训练
通过上面的方式，我们已经把数据特征和标签都已经制作完毕，接下来就是建立RNN模型进行训练了，这里使用tensorflow来建立模型。

### 合并上述的提取方法，通过数据生成器进行获得

In [3]:
register=pd.read_csv('user_register_log.txt',sep='\t',names=['user_id','register_day','register_type','device_type'])
launch=pd.read_csv('app_launch_log.txt',sep='\t',names=['user_id','launch_day'])
create=pd.read_csv('video_create_log.txt',sep='\t',names=['user_id','create_day'])
activity=pd.read_csv('user_activity_log.txt',sep='\t',names=['user_id','act_day','page','video_id','author_id','act_type'])

data_generator=DataGenerator(register,launch,create,activity)

### 构建RNN网络模型并进行训练

In [4]:
n_features = 12
n_hu = 8
with tf.variable_scope('train'):     # tf.variable_scope用来指定变量的作用域
    
    # 变量与输入
    lr = tf.placeholder(tf.float32, [], name='learning_rate')    # 定义学习率
    
    # 隐藏层到输出层的参数w, b    w_shape(n_hu,1)   b_shape(1)  n_huWie隐藏单元的个数
    W_out = tf.get_variable('W_out', [n_hu, 1])   
    b_out = tf.get_variable('b_out', [1])
    
    # x和y  x_shape(batch_size, seq_length, n_features)
    x = tf.placeholder(tf.float32, [None, None, n_features])
    y = tf.placeholder(tf.float32, [None, None])
    
    # batch_size和seq_length的大小
    batch_size = tf.shape(x)[0]
    seq_length = tf.shape(x)[1]
    
    # RNN 层
    cell = tf.nn.rnn_cell.GRUCell(n_hu)     # n_hu表示每个GRUcell里面的单元个数
    initial_state = cell.zero_state(batch_size, dtype=tf.float32)   # 指定初识状态，因为之前没有训练过
    outputs, state = tf.nn.dynamic_rnn(cell, x, initial_state=initial_state)  # 使用的动态Rnn
    # outputs(batch_size, max_seq_length, n_hu)     这是所有时间步的输出
    # state (batch_size, n_hu)   这是最后一个时间步的输出
    # 具体：https://blog.csdn.net/u010960155/article/details/81707498
    
    # 输出层
    outputs = tf.reshape(outputs, [-1, n_hu])    # （batch_size*max_seq_length, n_hu）
    logits = tf.matmul(outputs, W_out) + b_out    # (batch_size*max_seq_length)
    logits = tf.reshape(logits, tf.stack([batch_size, seq_length]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [16]:
# 选择部分预测结果与标签当做训练损失计算
logits_local_train = logits[:, :-14] # 这里-14或者是更小，因为本地训练，我们用前16天训练，16-23天测试。
label_local_train = y[:, :-14]

In [17]:
# 设置损失函数

# 正则化项
regularizer = tf.contrib.layers.l2_regularizer(0.00001)
penalty = tf.contrib.layers.apply_regularization(regularizer, tf.trainable_variables())

obj_local = tf.losses.sigmoid_cross_entropy(label_local_train, logits_local_train) + penalty
optimizer = tf.train.AdamOptimizer(lr)
set_local = optimizer.minimize(obj_local)

# 选择部分预测结果与标签当做测试损失计算
logits_local_test = logits[:, -8]
label_local_test = y[:, -8]   #  这里也可以选择其他的天

In [18]:
def train(n_obs=1000, step=1000, lr_feed=0.01):
    #date_seq = [31] + list(range(2, 16)) + [16] * 15
    variables = [set_local, obj_local, label_local_train, logits_local_train]
    
    for i in range(step):
        length, id_list, data_x, data_y = data_generator.next_batch(n_obs)
        _, los, lab, log = sess.run(variables, 
                                   feed_dict={x:data_x, y:data_y, lr:lr_feed})

这里这个取数据的思路得说一下： 也就是next_batch操作
这里取数据的时候，这个序列长度并不是随机取的。 如果看next_batch函数的时候就会注意到， 首先是先有了local_random_list， 然后从这里面取序列的长度， 那么这个local_random_list做了个什么事情呢？  他不是1-30的序列长度随机选，因为不同序列长度的用户个数不一样，而大部分应该集中在15天及以后，1-14天的那种用户可能很少，这样就有可能不够一个batch_size。 所以尽量的选15天以后的，这个local_random_list里面的元素就是从15开始，然后越往后的元素个数越多，那么相应被选择的几率就越大。 这样无非就是选择序列的时候，对应的用户让他多一些，这样好取够batch_size。<br><br>
然后有了这个序列长度，我们就可以取相应的用户了，pointer指针就是为了从头开始选择用户数据，够一个batch_size为止，如果超了用户数目的长度，就回到头上，然后打乱用户数据，再取。 （这里建议看一下代码，只描述可能不知道咋回事， 在deep.tools.py）  

In [19]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [20]:
train(n_obs=1000, step=2000, lr_feed=0.01)

### 模型测试并输出结果
f1值进行评估<br>
![](img/5.png)

In [21]:
def test():
    n_NA = 14     # 本地训练， 我们是1-16天训练，16-23天预测，所以这个地方最大是30-14
    # 优化目标和数据
    variables_1 = [obj_local, logits_local_train, label_local_train]
    variables_2 = [logits_local_test, label_local_test]
    
    obs_count, cum_loss, correct = 0, 0, 0
    user, prob, real = [], [], []
    
    # 训练损失
    for length, id_list, data_x, data_y in zip(*data_generator.get_set('train')):
        _obj, _logits_train, _label_train = sess.run(variables_1,
                                                    feed_dict={x:data_x, y:data_y, lr:0.001})
        obs_count += (length - n_NA) * len(id_list)
        cum_loss += _obj * (length - n_NA) * len(id_list)
        correct += np.sum((1 * (_logits_train>0) == _label_train))
    
    # 测试损失
    for length, id_list, data_x, data_y in zip(*data_generator.get_set('test')):
        _ = sess.run(variables_2, feed_dict={x:data_x, y:data_y, lr:0.001})
        _logits_test, _label_test = _
        real += list(_label_test)
        
        user += list(id_list)
        prob += list(1 / (1+np.exp(-_logits_test.reshape([-1]))))
    
    # 打印训练损失
    print('train_loss', cum_loss/obs_count)
    
    # 测试损失
    result = pd.DataFrame({'user_id':user, 'prob':prob, 'label':real})
    print('test_score:', f(result))
    
    return result

In [22]:
test()

train_loss 0.42956960269747707
test_score: [0.8053140974078207, 0.8054470655626724, 0.8050643507376791, 0.8053365676167374, 0.8047954268776796, 0.8040700300204564]


,user_id,prob,label
0,1274576,0.857779,1.0
1,109973,0.815846,1.0
2,134299,0.815846,0.0
3,1005835,0.595065,1.0
4,864582,0.820167,1.0
5,1293122,0.339015,1.0
6,896835,0.753712,1.0
7,630631,0.870561,1.0
8,1121791,0.753712,1.0
9,1038314,0.725725,1.0


### 余弦退火 + 热重启训练思想
越来越接近loss值的全局最小值时，学习率应该变得越来越小来使得模型尽可能接近最小值。 然后突然提高学习率，来“跳出”局部最小值并找到通向全局最小值的路径。 

In [26]:
def cos_annealing_local(epoch=5):
    all_result=None
    for i in range(epoch):
        train(step=2000,lr_feed=0.01)
        train(step=2000,lr_feed=0.001)
        result=test()
        print(sess.run(penalty))
        result.columns=['label','prob%s'%i,'user_id']
        if i==0:
            all_result=result
        else:
            all_result=pd.merge(all_result,result)
    return all_result

In [27]:
cos_annealing_local()

train_loss 0.4289520332444526
test_score: [0.8051643803524481, 0.8053407190635452, 0.8052916163578141, 0.8053609664543075, 0.8051175877262833, 0.8046993632948825]
0.0006277356
train_loss 0.4288679725521854
test_score: [0.8057853389521876, 0.8057455898927944, 0.8056542683408355, 0.8056776117813108, 0.8056491734874018, 0.8052826542116283]
0.00073696766
train_loss 0.42884548220977986
test_score: [0.8056395288231003, 0.805784473255997, 0.8058009228740936, 0.8053310723086299, 0.8054410110855245, 0.8056010965678502]
0.0008184217
train_loss 0.4286306096566724
test_score: [0.8057680836164639, 0.8055208333333332, 0.8058191349934469, 0.806068427845234, 0.8057995555085194, 0.8058112157534247]
0.000890288
train_loss 0.42857027057252495
test_score: [0.8060728326091454, 0.8059306190278355, 0.8060544271037566, 0.8058558085843216, 0.8056954515491102, 0.8058762364358654]
0.00093093584


,label,prob0,user_id,prob1,prob2,prob3,prob4
0,1274576,0.856154,1.0,0.840750,0.849629,0.849199,0.851723
1,109973,0.815051,1.0,0.805713,0.812808,0.815035,0.813512
2,134299,0.815051,0.0,0.805713,0.812808,0.815035,0.813512
3,1005835,0.586597,1.0,0.560536,0.565540,0.566560,0.570088
4,864582,0.828008,1.0,0.821973,0.824899,0.831624,0.826582
5,1293122,0.334039,1.0,0.323247,0.324855,0.324347,0.333743
6,896835,0.759586,1.0,0.746464,0.755796,0.762776,0.764424
7,630631,0.875573,1.0,0.872625,0.876371,0.883951,0.881113
8,1121791,0.759586,1.0,0.746464,0.755796,0.762776,0.764424
9,1038314,0.719833,1.0,0.701582,0.709675,0.716737,0.718184


## 知识点和思路的总结
1. pd.iterrows()函数
2. pd.groupby()函数

### pd.iterrows()函数
iterrows() 是在DataFrame中的行进行迭代的一个生成器，它返回每行的索引及一个包含行本身的对象。<br><br>
所以，当我们在需要遍历行数据的时候，就可以使用 iterrows()方法实现了。

In [31]:
df = pd.DataFrame(np.random.randn(3, 4), columns=list('ABCD'))
df

,A,B,C,D
0,-1.083707,0.856265,-0.254691,0.238692
1,-0.930010,0.969821,-0.912533,1.141066
2,0.132397,0.958107,0.004362,-1.627512


In [37]:
# 遍历行
for index, row in df.iterrows():    # index表示索引， row是一个Series结构，可以通过列名或者列索引来获取每一个元素
    print(index)
    print(row['A'])           # 这样是第一列的数据
    print(row[-1])   # 最后一列的数据
    print(row[1])    # 第二列的数据

0
-1.083706901708258
0.23869180151476993
0.8562654567977414
1
-0.930009726608264
1.1410662746749882
0.9698209735433845
2
0.1323970363518363
-1.6275121717858587
0.9581073260783397


### pd.groupby函数
这个函数的功能非常强大，类似于sql的groupby函数，对数据按照某一标准进行分组，然后进行一些统计。任何groupby操作都会涉及到下面的三个操作之一：
* Splitting：分割数据
* Applying：应用一个函数
* Combining:合并结果

在许多情况下，我们将数据分成几组，并在每个子集上应用一些功能。在应用中，我们可以执行以下操作：
* Aggregation ：计算一些摘要统计
* Transformation ：执行一些特定组的操作
* Filtration：根据某些条件下丢弃数据

参考博客：
* [Pandas之groupby( )用法笔记](https://www.cnblogs.com/bjwu/p/8970818.html)
* [Pandas分组运算（groupby）修炼](https://www.cnblogs.com/lemonbit/p/6810972.html)

In [40]:
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}
df = pd.DataFrame(ipl_data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


#### Pandas对象可以拆分为任何对象。分割对象的方法有多种：
* obj.groupby('key')
* obj.groupby(['key1','key2'])
* obj.groupby(key,axis=1)

现在让我们看看如何将分组对象应用于DataFrame对象


In [41]:
df.groupby('Team')

In [76]:
# 查看分组
df.groupby('Team').groups   # 第几个是

{'Devils': Int64Index([2, 3], dtype='int64'),
 'Kings': Int64Index([4, 6, 7], dtype='int64'),
 'Riders': Int64Index([0, 1, 8, 11], dtype='int64'),
 'Royals': Int64Index([9, 10], dtype='int64'),
 'kings': Int64Index([5], dtype='int64')}

In [43]:
# 根据多列进行分组
df.groupby(['Team', 'Year']).groups

{('Devils', 2014): Int64Index([2], dtype='int64'),
 ('Devils', 2015): Int64Index([3], dtype='int64'),
 ('Kings', 2014): Int64Index([4], dtype='int64'),
 ('Kings', 2016): Int64Index([6], dtype='int64'),
 ('Kings', 2017): Int64Index([7], dtype='int64'),
 ('Riders', 2014): Int64Index([0], dtype='int64'),
 ('Riders', 2015): Int64Index([1], dtype='int64'),
 ('Riders', 2016): Int64Index([8], dtype='int64'),
 ('Riders', 2017): Int64Index([11], dtype='int64'),
 ('Royals', 2014): Int64Index([9], dtype='int64'),
 ('Royals', 2015): Int64Index([10], dtype='int64'),
 ('kings', 2015): Int64Index([5], dtype='int64')}

In [86]:
# 遍历分组
grouped = df.groupby('Rank', as_index=False)
for name in grouped:
    print(name)

(1,       Team  Rank  Year  Points
0   Riders     1  2014     876
6    Kings     1  2016     756
7    Kings     1  2017     788
10  Royals     1  2015     804)
(2,       Team  Rank  Year  Points
1   Riders     2  2015     789
2   Devils     2  2014     863
8   Riders     2  2016     694
11  Riders     2  2017     690)
(3,      Team  Rank  Year  Points
3  Devils     3  2015     673
4   Kings     3  2014     741)
(4,      Team  Rank  Year  Points
5   kings     4  2015     812
9  Royals     4  2014     701)


#### 获取某一分组  get_group方法

In [45]:
# 获取某一分组
grouped = df.groupby('Year')
print(grouped.get_group(2014))

     Team  Rank  Year  Points
0  Riders     1  2014     876
2  Devils     2  2014     863
4   Kings     3  2014     741
9  Royals     4  2014     701


#### Aggregations（聚合）
聚合函数返回每个组的单个聚合值。一旦创建了group by对象，就可以对分组数据执行多个聚合操作。

In [87]:
"""agg方法实现聚合, 相比于apply，可以同时传入多个统计函数"""

# 针对同一列使用不同的统计方法 
grouped = df.groupby('Year', as_index=False)    # 这个as_index属性，如果是False，就是SQL风格的统计输出，如果是True，默认第一列变成了索引
print(grouped['Points'].agg({'mean':np.mean, 'std':np.std, 'max':np.max}))

# 针对不同的列使用不同的统计方法
print(grouped.agg({'Points':[np.mean, 'sum'], 'Rank':[np.max]}))

# 使用apply的话
print(grouped['Points'].apply(np.mean))
grouped.apply(lambda x: print(x))

   Year    mean        std  max
0  2014  795.25  87.439026  876
1  2015  769.50  65.035888  812
2  2016  725.00  43.840620  756
3  2017  739.00  69.296465  788
   Year  Points       Rank
           mean   sum amax
0  2014  795.25  3181    4
1  2015  769.50  3078    4
2  2016  725.00  1450    2
3  2017  739.00  1478    2
0    795.25
1    769.50
2    725.00
3    739.00
dtype: float64
     Team  Rank  Year  Points
0  Riders     1  2014     876
2  Devils     2  2014     863
4   Kings     3  2014     741
9  Royals     4  2014     701
     Team  Rank  Year  Points
0  Riders     1  2014     876
2  Devils     2  2014     863
4   Kings     3  2014     741
9  Royals     4  2014     701
      Team  Rank  Year  Points
1   Riders     2  2015     789
3   Devils     3  2015     673
5    kings     4  2015     812
10  Royals     1  2015     804
     Team  Rank  Year  Points
6   Kings     1  2016     756
8  Riders     2  2016     694
      Team  Rank  Year  Points
7    Kings     1  2017     788
11  Ride

""


In [61]:
"""查看每个组大小的另一种方法是应用size()函数"""

grouped = df.groupby('Team')
print(grouped.size())
print(grouped.count())
print(grouped.agg(np.size))

Team
Devils    2
Kings     3
Riders    4
Royals    2
kings     1
dtype: int64
        Rank  Year  Points
Team                      
Devils     2     2       2
Kings      3     3       3
Riders     4     4       4
Royals     2     2       2
kings      1     1       1
        Rank  Year  Points
Team                      
Devils     2     2       2
Kings      3     3       3
Riders     4     4       4
Royals     2     2       2
kings      1     1       1


#### Transformations
前面进行聚合运算的时候，得到的结果是一个以分组名为 index 的结果对象。如果我们想使用原数组的 index 的话，就需要进行 merge 转换。transform(func, args, *kwargs) 方法简化了这个过程，它会把 func 参数应用到所有分组，然后把结果放置到原数组的 index 上（如果结果是一个标量，就进行广播）：

In [63]:
grouped = df.groupby('Team')
score = lambda x: (x - x.mean()) / x.std()*10
print(grouped.transform(score))   # 应用于原数组的index上

         Rank       Year     Points
0  -15.000000 -11.618950  12.843272
1    5.000000  -3.872983   3.020286
2   -7.071068  -7.071068   7.071068
3    7.071068   7.071068  -7.071068
4   11.547005 -10.910895  -8.608621
5         NaN        NaN        NaN
6   -5.773503   2.182179  -2.360428
7   -5.773503   8.728716  10.969049
8    5.000000   3.872983  -7.705963
9    7.071068  -7.071068  -7.071068
10  -7.071068   7.071068   7.071068
11   5.000000  11.618950  -8.157595


In [65]:
print(grouped.transform(lambda x: print(x)))    

2    2
3    3
Name: Rank, dtype: int64
2    2014
3    2015
Name: Year, dtype: int64
2    863
3    673
Name: Points, dtype: int64
   Rank  Year  Points
2     2  2014     863
3     3  2015     673
4    3
6    1
7    1
Name: Rank, dtype: int64
4    2014
6    2016
7    2017
Name: Year, dtype: int64
4    741
6    756
7    788
Name: Points, dtype: int64
0     1
1     2
8     2
11    2
Name: Rank, dtype: int64
0     2014
1     2015
8     2016
11    2017
Name: Year, dtype: int64
0     876
1     789
8     694
11    690
Name: Points, dtype: int64
9     4
10    1
Name: Rank, dtype: int64
9     2014
10    2015
Name: Year, dtype: int64
9     701
10    804
Name: Points, dtype: int64
5    4
Name: Rank, dtype: int64
5    2015
Name: Year, dtype: int64
5    812
Name: Points, dtype: int64
    Rank  Year Points
0   None  None   None
1   None  None   None
2   None  None   None
3   None  None   None
4   None  None   None
5   None  None   None
6   None  None   None
7   None  None   None
8   None  None   None

In [64]:
print(grouped.agg(lambda x: print(x)))

2    2
3    3
Name: Rank, dtype: int64
4    3
6    1
7    1
Name: Rank, dtype: int64
0     1
1     2
8     2
11    2
Name: Rank, dtype: int64
9     4
10    1
Name: Rank, dtype: int64
5    4
Name: Rank, dtype: int64
2    2014
3    2015
Name: Year, dtype: int64
4    2014
6    2016
7    2017
Name: Year, dtype: int64
0     2014
1     2015
8     2016
11    2017
Name: Year, dtype: int64
9     2014
10    2015
Name: Year, dtype: int64
5    2015
Name: Year, dtype: int64
2    863
3    673
Name: Points, dtype: int64
4    741
6    756
7    788
Name: Points, dtype: int64
0     876
1     789
8     694
11    690
Name: Points, dtype: int64
9     701
10    804
Name: Points, dtype: int64
5    812
Name: Points, dtype: int64
        Rank  Year Points
Team                     
Devils  None  None   None
Kings   None  None   None
Riders  None  None   None
Royals  None  None   None
kings   None  None   None


In [68]:
print(grouped.apply(lambda x: print(x)))

     Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673
     Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673
    Team  Rank  Year  Points
4  Kings     3  2014     741
6  Kings     1  2016     756
7  Kings     1  2017     788
      Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
8   Riders     2  2016     694
11  Riders     2  2017     690
      Team  Rank  Year  Points
9   Royals     4  2014     701
10  Royals     1  2015     804
    Team  Rank  Year  Points
5  kings     4  2015     812
Empty DataFrame
Columns: []
Index: []


使用apply()处理的对象是一个个的类如DataFrame的数据表，然而agg()则每次只传入一列,从列的角度进行输出。

#### Filtration
过滤数据

In [70]:
grouped.size()

Team
Devils    2
Kings     3
Riders    4
Royals    2
kings     1
dtype: int64

In [69]:
print(df.groupby('Team').filter(lambda x: len(x) >= 3))

      Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
4    Kings     3  2014     741
6    Kings     1  2016     756
7    Kings     1  2017     788
8   Riders     2  2016     694
11  Riders     2  2017     690
